In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
X = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [8]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00


In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

# Without Hyperparameter Tuning

In [10]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 18ms/step - loss: 0.6960 - accuracy: 0.5798 - val_loss: 0.6850 - val_accuracy: 0.6299
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6417 - accuracy: 0.6808 - val_loss: 0.6400 - val_accuracy: 0.7013
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6012 - accuracy: 0.7166 - val_loss: 0.6063 - val_accuracy: 0.7143
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5685 - accuracy: 0.7150 - val_loss: 0.5784 - val_accuracy: 0.7273
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5431 - accuracy: 0.7296 - val_loss: 0.5579 - val_accuracy: 0.7597
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5233 - accuracy: 0.7378 - val_loss: 0.5422 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5081 - accuracy: 0.7524 - val_loss: 0.5313 - val_accuracy: 0.7857
Epoch 8/100


# Keras hyperparameter tuner
1. Select appropriate optimizer
2. No. of nodes in a layer
3. No. to hidden layers
4. All in one model

## Tuning optimizer

In [12]:
import keras_tuner as kt

In [13]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  optimizer = hp.Choice('optimizer', ['SGD','RMSprop','Adam','AdamW',
                                      'Adadelta','Adagrad','Adamax','Adafactor',
                                      'Nadam','Ftrl','Lion','Loss Scale Optimizer'])

  model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

  return model

In [14]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='keras tuner',
                        project_name='optimizer tuning')

In [15]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.37662336230278015

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 23s


In [16]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'RMSprop'}

In [17]:
model = tuner.get_best_models(num_models=1)[0]

In [18]:
model.fit(X_train, y_train, epochs=100, initial_epoch=10, validation_data=(X_test, y_test))

Epoch 11/100
20/20 [==============================] - 1s 17ms/step - loss: 0.4569 - accuracy: 0.7752 - val_loss: 0.5044 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4531 - accuracy: 0.7736 - val_loss: 0.5025 - val_accuracy: 0.7727
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4495 - accuracy: 0.7752 - val_loss: 0.5035 - val_accuracy: 0.7857
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4464 - accuracy: 0.7752 - val_loss: 0.5025 - val_accuracy: 0.7662
Epoch 15/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4444 - accuracy: 0.7752 - val_loss: 0.5023 - val_accuracy: 0.7662
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4419 - accuracy: 0.7720 - val_loss: 0.5039 - val_accuracy: 0.7597
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4399 - accuracy: 0.7769 - val_loss: 0.5034 - val_accuracy: 0.7597
Epoch

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Tuning layer nodes

In [20]:
def build_model(hp):
  model = Sequential()
  nodes = hp.Int('Nodes', min_value=8, max_value=128, step=8)

  model.add(Dense(units=nodes, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [21]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='keras tuner',
                        project_name='layer nodes tuning')

In [22]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 26s


In [23]:
tuner.get_best_hyperparameters()[0].values

{'Nodes': 72}

In [24]:
model = tuner.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 1)                 73        
                                                                 
Total params: 721 (2.82 KB)
Trainable params: 721 (2.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
model.fit(X_train, y_train, epochs=100, initial_epoch=10, validation_data=(X_test, y_test))

Epoch 11/100
20/20 [==============================] - 1s 16ms/step - loss: 0.4572 - accuracy: 0.7655 - val_loss: 0.4898 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4534 - accuracy: 0.7752 - val_loss: 0.4906 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4498 - accuracy: 0.7785 - val_loss: 0.4922 - val_accuracy: 0.7727
Epoch 14/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4478 - accuracy: 0.7769 - val_loss: 0.4957 - val_accuracy: 0.7792
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4456 - accuracy: 0.7769 - val_loss: 0.4990 - val_accuracy: 0.7662
Epoch 16/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4454 - accuracy: 0.7769 - val_loss: 0.4983 - val_accuracy: 0.7857
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4416 - accuracy: 0.7769 - val_loss: 0.4989 - val_accuracy: 0.7662
Epoch

## Tuning no. of layers

In [27]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(96, activation='relu', input_dim=8))

  for i in range(hp.Int('layers', min_value=1, max_value=10)):
    model.add(Dense(96, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [28]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='keras tuner',
                        project_name='number of layers tuning')

In [29]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 44s


In [30]:
tuner.get_best_hyperparameters()[0].values

{'layers': 1}

In [31]:
model = tuner.get_best_models(num_models=1)[0]

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 96)                864       
                                                                 
 dense_1 (Dense)             (None, 96)                9312      
                                                                 
 dense_2 (Dense)             (None, 1)                 97        
                                                                 
Total params: 10273 (40.13 KB)
Trainable params: 10273 (40.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
model.fit(X_train, y_train, epochs=100, initial_epoch=10, validation_data=(X_test, y_test))

Epoch 11/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4731 - accuracy: 0.7785 - val_loss: 0.4898 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4506 - accuracy: 0.7883 - val_loss: 0.4920 - val_accuracy: 0.7727
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4407 - accuracy: 0.7899 - val_loss: 0.5026 - val_accuracy: 0.7727
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4315 - accuracy: 0.7948 - val_loss: 0.5074 - val_accuracy: 0.7662
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4233 - accuracy: 0.8062 - val_loss: 0.5150 - val_accuracy: 0.7662
Epoch 16/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4216 - accuracy: 0.8029 - val_loss: 0.5117 - val_accuracy: 0.7662
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4151 - accuracy: 0.8046 - val_loss: 0.5107 - val_accuracy: 0.7792
Epoch

## All in one model

In [34]:
def build_model(hp):
  model = Sequential()

  counter = 0
  activations = ['relu','sigmoid','softmax','softplus','softsign','tanh','selu','elu','exponential',
                 'leaky_relu','relu6','silu','hard_silu','gelu','hard_sigmoid','linear','mish','log_softmax']
  dropout_rates = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

  for i in range(hp.Int('layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(Dense(units=hp.Int(f'node_{i}', min_value=8, max_value=128, step=128),
                      activation=hp.Choice(f'activation_{i}', activations),
                      input_dim=8))
      if hp.Boolean('dropout'):
        model.add(Dropout(hp.Choice(f'dropout_{i}', dropout_rates)))

    else:
      model.add(Dense(units=hp.Int(f'node_{i}', min_value=8, max_value=128, step=128),
                      activation=hp.Choice(f'activation_{i}', activations)))
      if hp.Boolean('dropout'):
        model.add(Dropout(hp.Choice(f'dropout_{i}', dropout_rates)))

    counter+=1

  model.add(Dense(1, activation='sigmoid'))

  optimizer = hp.Choice('optimizer', ['SGD','RMSprop','Adam','AdamW',
                                      'Adadelta','Adagrad','Adamax','Adafactor',
                                      'Nadam','Ftrl','Lion','Loss Scale Optimizer'])

  model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

  return model

In [35]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='keras tuner',
                        project_name='all in one tuning')

In [36]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.6298701167106628

Best val_accuracy So Far: 0.7077922224998474
Total elapsed time: 00h 00m 30s


In [37]:
tuner.get_best_hyperparameters()[0].values

{'layers': 10,
 'node_0': 8,
 'activation_0': 'gelu',
 'dropout': True,
 'optimizer': 'Adam',
 'dropout_0': 0.1,
 'node_1': 8,
 'activation_1': 'relu',
 'dropout_1': 0.1,
 'node_2': 8,
 'activation_2': 'relu',
 'dropout_2': 0.1,
 'node_3': 8,
 'activation_3': 'relu',
 'dropout_3': 0.1,
 'node_4': 8,
 'activation_4': 'relu',
 'dropout_4': 0.1,
 'node_5': 8,
 'activation_5': 'relu',
 'dropout_5': 0.1,
 'node_6': 8,
 'activation_6': 'relu',
 'dropout_6': 0.1,
 'node_7': 8,
 'activation_7': 'relu',
 'dropout_7': 0.1,
 'node_8': 8,
 'activation_8': 'relu',
 'dropout_8': 0.1,
 'node_9': 8,
 'activation_9': 'relu',
 'dropout_9': 0.1}

In [38]:
model = tuner.get_best_models(num_models=1)[0]

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dropout_2 (Dropout)         (None, 8)                 0         
                                                                 
 dense_3 (Dense)             (None, 8)                 7

In [40]:
model.fit(X_train, y_train, epochs=100, initial_epoch=10, validation_data=(X_test, y_test))

Epoch 11/100
20/20 [==============================] - 4s 21ms/step - loss: 0.5778 - accuracy: 0.7101 - val_loss: 0.5527 - val_accuracy: 0.7143
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5771 - accuracy: 0.7199 - val_loss: 0.5465 - val_accuracy: 0.7208
Epoch 13/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5688 - accuracy: 0.7134 - val_loss: 0.5430 - val_accuracy: 0.7143
Epoch 14/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5540 - accuracy: 0.7215 - val_loss: 0.5419 - val_accuracy: 0.7078
Epoch 15/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5666 - accuracy: 0.7101 - val_loss: 0.5501 - val_accuracy: 0.7013
Epoch 16/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5449 - accuracy: 0.7264 - val_loss: 0.5372 - val_accuracy: 0.7013
Epoch 17/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5342 - accuracy: 0.7378 - val_loss: 0.5295 - val_accuracy: 0.7078
Epoch

In [41]:
!zip -r keras-tuner.zip keras-tuner/

  adding: keras-tuner/ (stored 0%)
  adding: keras-tuner/layer nodes tuning/ (stored 0%)
  adding: keras-tuner/layer nodes tuning/trial_03/ (stored 0%)
  adding: keras-tuner/layer nodes tuning/trial_03/checkpoint (deflated 48%)
  adding: keras-tuner/layer nodes tuning/trial_03/trial.json (deflated 56%)
  adding: keras-tuner/layer nodes tuning/trial_03/checkpoint.index (deflated 55%)
  adding: keras-tuner/layer nodes tuning/trial_03/checkpoint.data-00000-of-00001 (deflated 14%)
  adding: keras-tuner/layer nodes tuning/trial_00/ (stored 0%)
  adding: keras-tuner/layer nodes tuning/trial_00/checkpoint (deflated 48%)
  adding: keras-tuner/layer nodes tuning/trial_00/trial.json (deflated 56%)
  adding: keras-tuner/layer nodes tuning/trial_00/checkpoint.index (deflated 55%)
  adding: keras-tuner/layer nodes tuning/trial_00/checkpoint.data-00000-of-00001 (deflated 10%)
  adding: keras-tuner/layer nodes tuning/trial_02/ (stored 0%)
  adding: keras-tuner/layer nodes tuning/trial_02/checkpoint (